In [ ]:
### Eval script for deuteron GFMC deformation.

import argparse
import analysis as al
import getpass
import matplotlib.pyplot as plt
import numpy as onp
import scipy
import scipy.interpolate
import scipy.integrate
import jax.scipy
import jax.scipy.special
import pickle
import paper_plt
import tqdm.auto as tqdm
import afdmc_lib_col as adl
import os
import pickle
from afdmc_lib_col import NI,NS,mp_Mev,fm_Mev
import jax
import jax.numpy as np
import sys
from itertools import repeat
import time
import h5py
import math
import mpmath
from functools import partial

from itertools import permutations
import torch
import torch.nn as nn

onp.random.seed(0)

paper_plt.load_latex_config()

parser = argparse.ArgumentParser()
parser.add_argument('--n_walkers', type=int, default=1000)
parser.add_argument('--dtau_iMev', type=float, required=True)
parser.add_argument('--n_step', type=int, required=True)
parser.add_argument('--n_skip', type=int, default=100)
parser.add_argument('--resampling', type=int, default=None)
parser.add_argument('--alpha', type=float, default=1)
parser.add_argument('--mu', type=float, default=1.0)
parser.add_argument('--mufac', type=float, default=1.0)
parser.add_argument('--Nc', type=int, default=3)
parser.add_argument('--N_coord', type=int, default=3)
parser.add_argument('--nf', type=int, default=5)
parser.add_argument('--OLO', type=str, default="LO")
parser.add_argument('--spoila', type=float, default=1)
parser.add_argument('--afac', type=float, default=1)
parser.add_argument('--spoilf', type=str, default="hwf")
parser.add_argument('--outdir', type=str, required=True)
parser.add_argument('--input_Rs_database', type=str, default="")
parser.add_argument('--log_mu_r', type=float, default=1)
parser.add_argument('--cutoff', type=float, default=0.0)
parser.add_argument('--L', type=float, default=0.0)
parser.add_argument('--Lcut', type=int, default=5)
parser.add_argument('--spoilS', type=float, default=1)
parser.add_argument('--wavefunction', type=str, default="compact")
parser.add_argument('--potential', type=str, default="full")
parser.add_argument('--spoilaket', type=float, default=1)
parser.add_argument('--masses', type=float, default=0., nargs='+')
parser.add_argument('--verbose', dest='verbose', action='store_true', default=False)
globals().update(vars(parser.parse_args()))

#######################################################################################

In [ ]:
#By default, Nc=3 and nf=5

CF = (Nc**2 - 1)/(2*Nc)
VB = alpha*CF/(Nc-1)
SingC3 = -(Nc+1)/8

In [ ]:
# imaginary time points for GFMC evolution
tau_iMev = dtau_iMev * n_step
xs = np.linspace(0, tau_iMev, endpoint=True, num=n_step+1)

beta0 = 11/3*Nc - 2/3*nf
beta1 = 34/3*Nc**2 - 20/3*Nc*nf/2 - 2*CF*nf
beta2 = 2857/54*Nc**3 + CF**2*nf-205/9*Nc*CF*nf/2-1415/27*Nc**2*nf/2+44/9*CF*(nf/2)**2+158/27*Nc*(nf/2)**2
aa1 = 31/9*Nc-10/9*nf
zeta3 = scipy.special.zeta(3)
zeta5 = scipy.special.zeta(5)
zeta51 = 1/2 + 1/3 + 1/7 + 1/51 + 1/4284
zeta6 = scipy.special.zeta(6)
aa2 = ( 4343/162 + 6*np.pi**2 - np.pi**4/4 + 22/3*zeta3 )*Nc**2 - ( 1798/81 + 56/3*zeta3 )*Nc*nf/2 - ( 55/3 - 16*zeta3  )*CF*nf/2 + (10/9*nf)**2
dFF = (18-Nc**2+Nc**4)/(96*Nc**2)
dFA = Nc*(Nc**2+6)/48
alpha4 = float(mpmath.polylog(4,1/2))*0+(-np.log(2))**4/(4*3*2*1)
ss6 = zeta51+zeta6
aa30 = dFA*( np.pi**2*( 7432/9-4736*alpha4+np.log(2)*(14752/3-3472*zeta3)-6616*zeta3/3)  +  np.pi**4*(-156+560*np.log(2)/3+496*np.log(2)**2/3)+1511*np.pi**6/45)  + Nc**3*(385645/2916 + np.pi**2*( -953/54 +584/3*alpha4 +175/2*zeta3 + np.log(2)*(-922/9+217*zeta3/3) ) +584*zeta3/3 + np.pi**4*( 1349/270-20*np.log(2)/9-40*np.log(2)**2/9 ) -1927/6*zeta5 -143/2*zeta3**2-4621/3024*np.pi**6+144*ss6  )
aa31 = dFF*( np.pi**2*(1264/9-976*zeta3/3+np.log(2)*(64+672*zeta3)) + np.pi**4*(-184/3+32/3*np.log(2)-32*np.log(2)**2) +10/3*np.pi**6 ) + CF**2/2*(286/9+296/3*zeta3-160*zeta5)+Nc*CF/2*(-71281/162+264*zeta3+80*zeta5)+Nc**2/2*(-58747/486+np.pi**2*(17/27-32*alpha4+np.log(2)*(-4/3-14*zeta3)-19/3*zeta3)-356*zeta3+np.pi**4*(-157/54-5*np.log(2)/9+np.log(2)**2)+1091*zeta5/6+57/2*zeta3**2+761*np.pi**6/2520-48*ss6)
aa32 = Nc/4*(12541/243+368/3*zeta3+64*np.pi**4/135)+CF/4*(14002/81-416*zeta3/3)
aa33 = -(20/9)**3*1/8
aa3 = aa30+aa31*nf+aa32*nf**2+aa33*nf**3

L = log_mu_r
VB_LO = VB
VB_NLO = VB * (1 + alpha/(4*np.pi)*(aa1 + 2*beta0*log_mu_r))
VB_NNLO = VB * (1 + alpha/(4*np.pi)*(aa1 + 2*beta0*L) + (alpha/(4*np.pi))**2*( beta0**2*(4*L**2 + np.pi**2/3) + 2*( beta1+2*beta0*aa1 )*L + aa2 ) )

In [ ]:
Rprime = lambda R: adl.norm_3vec(R)*np.exp(np.euler_gamma)*mu

In [ ]:
if OLO == "LO":
    @partial(jax.jit)
    def potential_fun(R):
            return -1*VB/adl.norm_3vec(R)
    @partial(jax.jit)
    def symmetric_potential_fun(R):
            return spoilS*(Nc - 1)/(Nc + 1)*VB/adl.norm_3vec(R)
    @partial(jax.jit)
    def singlet_potential_fun(R):
            return -1*(Nc - 1)*VB/adl.norm_3vec(R)
    @partial(jax.jit)
    def octet_potential_fun(R):
            return spoilS*(Nc - 1)/CF/(2*Nc)*VB/adl.norm_3vec(R)
            #return -1*(CF-Nc/2)*(Nc - 1)/CF*VB/adl.norm_3vec(R)
    @partial(jax.jit)
    def potential_fun_sum(R):
            return -1*VB*FV_Coulomb(R, L, nn)
    @partial(jax.jit)
    def symmetric_potential_fun_sum(R):
            return spoilS*(Nc - 1)/(Nc + 1)*VB*FV_Coulomb(R, L, nn)
    @partial(jax.jit)
    def singlet_potential_fun_sum(R):
            return -1*(Nc - 1)*VB*FV_Coulomb(R, L, nn)
    @partial(jax.jit)
    def octet_potential_fun_sum(R):
            return spoilS*(Nc - 1)/CF/(2*Nc)*VB*FV_Coulomb(R, L, nn)
elif OLO == "NLO":
    @partial(jax.jit)
    def potential_fun(R):
        return -1*VB/adl.norm_3vec(R)*(1 + alpha/(4*np.pi)*(2*beta0*np.log(Rprime(R))+aa1))
    @partial(jax.jit)
    def potential_fun_sum(R):
            return calculate_sum(potential_fun, R, L, nn)
    def symmetric_potential_fun(R):
        return (Nc - 1)/(Nc + 1)*VB*spoilS/adl.norm_3vec(R)*(1 + alpha/(4*np.pi)*(2*beta0*np.log(Rprime(R))+aa1))
    def symmetric_potential_fun_sum(R):
            return calculate_sum(symmetric_potential_fun, R, L, nn)
    @partial(jax.jit)
    def singlet_potential_fun(R):
        return -1*(Nc - 1)*VB/adl.norm_3vec(R)*(1 + alpha/(4*np.pi)*(2*beta0*np.log(Rprime(R))+aa1))
    @partial(jax.jit)
    def singlet_potential_fun_sum(R):
            return calculate_sum(potential_fun, R, L, nn)
    def octet_potential_fun(R):
        return spoilS*(Nc - 1)/CF/(2*Nc)*VB/adl.norm_3vec(R)*(1 + alpha/(4*np.pi)*(2*beta0*np.log(Rprime(R))+aa1))
    def octet_potential_fun_sum(R):
            return calculate_sum(symmetric_potential_fun, R, L, nn)
elif OLO == "NNLO":
    @partial(jax.jit)
    def potential_fun(R):
        return -1*spoilS*VB/adl.norm_3vec(R)*(1 + alpha/(4*np.pi)*(2*beta0*np.log(Rprime(R))+aa1) + (alpha/(4*np.pi))**2*( beta0**2*(4*np.log(Rprime(R))**2 + np.pi**2/3) + 2*( beta1+2*beta0*aa1 )*np.log(Rprime(R))+ aa2 + Nc*(Nc-2)/2*((np.pi)**4-12*(np.pi)**2) ) )
    @partial(jax.jit)
    def potential_fun_sum(R):
            return calculate_sum(potential_fun, R, L, nn)
    def symmetric_potential_fun(R):
        return spoilS*(Nc - 1)/(Nc + 1)*VB/adl.norm_3vec(R)*(1 + alpha/(4*np.pi)*(2*beta0*np.log(Rprime(R))+aa1) + (alpha/(4*np.pi))**2*( beta0**2*(4*np.log(Rprime(R))**2 + np.pi**2/3) + 2*( beta1+2*beta0*aa1 )*np.log(Rprime(R))+ aa2 + Nc*(Nc+2)/2*((np.pi)**4-12*(np.pi)**2) ) )
    def symmetric_potential_fun_sum(R):
            return calculate_sum(symmetric_potential_fun, R, L, nn)
    @partial(jax.jit)
    def singlet_potential_fun(R):
        return -1*(Nc - 1)*VB/adl.norm_3vec(R)*(1 + alpha/(4*np.pi)*(2*beta0*np.log(Rprime(R))+aa1) + (alpha/(4*np.pi))**2*( beta0**2*(4*np.log(Rprime(R))**2 + np.pi**2/3) + 2*( beta1+2*beta0*aa1 )*np.log(Rprime(R))+ aa2 ) )
    @partial(jax.jit)
    def singlet_potential_fun_sum(R):
            return calculate_sum(potential_fun, R, L, nn)
    def octet_potential_fun(R):
        return spoilS*(Nc - 1)/CF/(2*Nc)*VB/adl.norm_3vec(R)*(1 + alpha/(4*np.pi)*(2*beta0*np.log(Rprime(R))+aa1) + (alpha/(4*np.pi))**2*( beta0**2*(4*np.log(Rprime(R))**2 + np.pi**2/3) + 2*( beta1+2*beta0*aa1 )*np.log(Rprime(R))+ aa2 + (Nc**2)*((np.pi)**4-12*(np.pi)**2) ) )
    def octet_potential_fun_sum(R):
            return calculate_sum(symmetric_potential_fun, R, L, nn)
else:
        print("order not supported")
        throw(0)

### Code to define the laplacian in an efficient and fast way :

In [ ]:
@partial(jax.jit)
def laplacian_f_R(Rs, wavefunction=bra_wavefunction, a0=a0, afac=afac, masses=absmasses):
    #N_walkers = Rs.shape[0]
    #assert Rs.shape == (N_walkers, N_coord, 3)
    nabla_psi_tot = 0
    # terms where laplacian hits one piece of wvfn
    # laplacian hits r_kl
    for k in range(N_coord):
        for l in range(N_coord):
            if k!=l and l>=k:
                #if wavefunction == "product":
                #    baryon_0 = 1
                #    if k < N_coord/2:
                #        baryon_0 = 0
                #    baryon_1 = 1
                #    if l < N_coord/2:
                #        baryon_1 = 0
                #    if baryon_0 != baryon_1:
                #        continue
                # wvfn includes r_ij
                nabla_psi = 1
                for i in range(N_coord):
                    for j in range(N_coord):
                        thisa0 = a0
                        if i!=j and j>=i:
                            if wavefunction == "product":
                                baryon_0 = 1
                                if i < N_coord/2:
                                    baryon_0 = 0
                                baryon_1 = 1
                                if j < N_coord/2:
                                    baryon_1 = 0
                                if baryon_0 != baryon_1:
                                    thisa0 *= afac
                                    #continue
                            ri = Rs[...,i,:]
                            rj = Rs[...,j,:]
                            rij_norm = adl.norm_3vec(ri - rj)
                            mij = 2*masses[i]*masses[j]/(masses[i]+masses[j])
                            thisa0 /= mij
                            # nabla_k^2 r_kl = nabla_l^2 r_kl
                            # factor of two included to account for both terms appearing in laplacian
                            if k == i and l == j:
                                #nabla_psi = nabla_psi * (2/thisa0**2 - 4/(thisa0*rij_norm)) * np.exp(-rij_norm/thisa0)
                                nabla_psi = nabla_psi * ((1/thisa0**2 - 2/(thisa0*rij_norm))/masses[k] + (1/thisa0**2 - 2/(thisa0*rij_norm))/masses[l]) * np.exp(-rij_norm/thisa0)
                            else:
                                nabla_psi = nabla_psi * np.exp(-rij_norm/thisa0)
                nabla_psi_tot += nabla_psi
    # terms where gradients hit separate pieces of wvfn
    # laplacian involves particle a
    for a in range(N_coord):
        # first gradient involves r_kl
        for k in range(N_coord):
            for l in range(N_coord):
                if k!=l and l>=k and (a==k or a==l):
                    #if wavefunction == "product":
                    #    baryon_0 = 1
                    #    if k < N_coord/2:
                    #        baryon_0 = 0
                    #    baryon_1 = 1
                    #    if l < N_coord/2:
                    #        baryon_1 = 0
                    #    if baryon_0 != baryon_1:
                    #        continue
                    # second gradient involves r_mn
                    for m in range(N_coord):
                        for n in range(N_coord):
                            if m!=n and n>=m and (m!=k or n!=l) and (a==m or a==n):
                                # sum over the 3-d components of gradient
                                for x in range(3):
                                    # wvfn involves r_ij
                                    nabla_psi = 1
                                    for i in range(N_coord):
                                        for j in range(N_coord):
                                            thisa0 = a0
                                            if i!=j and j>=i:
                                                if wavefunction == "product":
                                                    baryon_0 = 1
                                                    if i < N_coord/2:
                                                        baryon_0 = 0
                                                    baryon_1 = 1
                                                    if j < N_coord/2:
                                                        baryon_1 = 0
                                                    if baryon_0 != baryon_1:
                                                        thisa0 *= afac
                                                        #continue
                                                ri = Rs[...,i,:]
                                                rj = Rs[...,j,:]
                                                rij_norm = adl.norm_3vec(ri - rj)
                                                mij = 2*masses[i]*masses[j]/(masses[i]+masses[j])
                                                thisa0 /= mij
                                                rsign = 0
                                                # grad_a r_ij = rsign * (ri - rj)
                                                if a == i:
                                                    rsign = 1
                                                elif a == j:
                                                    rsign = -1
                                                if (k == i and l == j) or (m == i and n == j):
                                                    nabla_psi = rsign * nabla_psi * (ri[:,x] - rj[:,x])/(thisa0*rij_norm) * np.exp(-rij_norm/thisa0) / masses[a]
                                                else:
                                                    nabla_psi = nabla_psi * np.exp(-rij_norm/thisa0)
                                    nabla_psi_tot += nabla_psi
    return nabla_psi_tot

### Translate the alpha_running.nb code into python

In [2]:
import numpy as np
import scipy.special as sp

# Zeta function using scipy
def Zeta(s):
    return sp.zeta(s, 1)  # '1' is the q parameter, for Riemann Zeta it's set to 1

Pi = np.pi

# Translating the formulas
def CF(N):
    CF = (N**2 - 1) / (2 * N)
    return CF

def CA(N):
    CA = N
    return CA

def dFA(N):
    dFA = N * (N**2 + 6) / 48
    return dFA

def dFF(N):
    dFF = (N**4 - 6 * N**2 + 18) / (96 * N**2)
    return dFF

def dAA(N):
    dAA = N**2 * (N**2 + 36) / 24
    return dAA
    
def beta0(N,Nf):
    beta0 = (1 / (4 * Pi)) * (11/3 * CA(N) - 2/3 * Nf)
    return beta0

def beta1(N,Nf):
    beta1 = (1 / (4 * Pi))**2 * (34/3 * CA(N)**2 - 10/3 * CA(N) * Nf - 2 * CF(N) * Nf)
    return beta1

def beta2(N,Nf):
    beta2 = (1 / (4 * Pi))**3 * (2857/54 * CA(N)**3 - 1415/27 * CA(N)**2 * Nf / 2 + 
         158/27 * CA(N) * Nf**2 / 4 + 44/9 * CF(N) * Nf**2 / 4 - 
         205/9 * CF(N) * CA(N) * Nf / 2 + CF(N)**2 * Nf)
    return beta2

def beta3(N,Nf):
    beta3 = (1 / (4 * Pi))**4 * (CA(N) * CF(N) * Nf**2 / 4 * (17152/243 + 448/9 * Zeta(3)) + 
         CA(N) * CF(N)**2 * Nf / 2 * (-4204/27 + 352/9 * Zeta(3)) + 
         424/243 * CA(N) * Nf**3 / 8 + 1232/243 * CF(N) * Nf**3 / 8 + 
         CA(N)**2 * CF(N) * Nf / 2 * (7073/243 - 656/9 * Zeta(3)) + 
         CA(N)**2 * Nf**2 / 4 * (7930/81 + 224/9 * Zeta(3)) + 
         CA(N)**3 * Nf / 2 * (-39143/81 + 136/3 * Zeta(3)) + 
         CA(N)**4 * (150653/486 - 44/9 * Zeta(3)) + 
         CF(N)**2 * Nf**2 / 4 * (1352/27 - 704/9 * Zeta(3)) + 
         46 * CF(N)**3 * Nf / 2 + 
         Nf * dFA(N) * (512/9 - 1664/3 * Zeta(3)) + 
         Nf**2 * dFF(N) * (-704/9 + 512/3 * Zeta(3)) + 
         dAA(N) * (-80/9 + 704/3 * Zeta(3)))
    return beta3


In [3]:
# LambdaQCD function
def LambdaQCD(f):
    if f == 3:
        return 0.338
    elif f == 4:
        return 0.296
    elif f == 5:
        return 0.213
    elif f == 6:
        return 0.090
    else:
        raise ValueError("Unknow value for f")

# LQ function
def LQ(Q, f):
    return np.log(Q ** 2 / LambdaQCD(f) ** 2)


In [4]:
# Constants
Mt = 173.34
Mb = 4.7
Mc = 1.5

def AlphaSNf(Q, f):
    LQ_val = LQ(Q, f)

    term1 = 1 / (beta0(3,f) * LQ_val)
    term2 = beta1(3,f) * np.log(LQ_val) / (beta0(3,f) ** 3 * LQ_val ** 2)
    term3 = 1 / (beta0(3,f) ** 3 * LQ_val ** 3) * (beta1(3,f) ** 2 / beta0(3,f) ** 2 * (np.log(LQ_val) ** 2 - np.log(LQ_val) - 1) + beta2(3,f) / beta0(3,f))
    term4 = 1 / (beta0(3,f) ** 4 * LQ_val ** 4) * (beta1(3,f) ** 3 / beta0(3,f) ** 3 * (-np.log(LQ_val) ** 3 + 5/2 * np.log(LQ_val) ** 2 + 2 * np.log(LQ_val) - 1/2))
    term5 = -1 / (beta0(3,f) ** 4 * LQ_val ** 4) * (3 * beta1(3,f) * beta2(3,f) * np.log(LQ_val) / beta0(3,f) ** 2 + beta3(3,f) / (2 * beta0(3,f)))

    return term1 - term2 + term3 + term4 - term5

def MAlphaS(Q, f):
    alpha_s_nf = AlphaSNf(Q, f)
    return 1 + (-0.291667) * (alpha_s_nf / Pi) ** 2 + (-5.32389 + (f - 1) * 0.26247) * (alpha_s_nf / Pi) ** 3


def AlphaS(Q):
    if Q < 1:
        return MAlphaS(Mc, 4) * MAlphaS(Mb, 5) * AlphaSNf(1, 3)
    elif 1 <= Q < Mc:
        return MAlphaS(Mc, 4) * MAlphaS(Mb, 5) * AlphaSNf(Q, 3)
    elif Mc <= Q < Mb:
        return MAlphaS(Mb, 5) * AlphaSNf(Q, 4)
    elif Mb <= Q < Mt:
        return AlphaSNf(Q, 5)
    else:  # Q >= Mt
        return (1 / MAlphaS(Mt, 6)) * AlphaSNf(Q, 6)


#### Alpha loops :

In [5]:
def AlphaSNf3Loop(Q, f):
    LQ_val = LQ(Q, f)
    term1 = 1 / (beta0(3,f) * LQ_val)
    term2 = beta1(3,f) * np.log(LQ_val) / (beta0(3,f) ** 3 * LQ_val ** 2)
    term3 = 1 / (beta0(3,f) ** 3 * LQ_val ** 3) * ((beta1(3,f) ** 2 / beta0(3,f) ** 2) * (np.log(LQ_val) ** 2 - np.log(LQ_val) - 1) + beta2(3,f) / beta0(3,f))
    return term1 - term2 + term3

def AlphaSNf2Loop(Q, f):
    LQ_val = LQ(Q, f)
    term1 = 1 / (beta0(3,f) * LQ_val)
    term2 = beta1(3,f) * np.log(LQ_val) / (beta0(3,f) ** 3 * LQ_val ** 2)
    return term1 - term2

def AlphaSNf1Loop(Q, f):
    LQ_val = LQ(Q, f)
    return 1 / (beta0(3,f) * LQ_val)


In [6]:
def MAlphaS2Loop(Q, f):
    alpha_s_nf = AlphaSNf(Q, f)
    return 1 + (-0.291667 ) * (alpha_s_nf / Pi) ** 2


def AlphaS3Loop(Q):
    if Q < 1:
        return MAlphaS2Loop(Mc, 4) * MAlphaS2Loop(Mb, 5) * AlphaSNf3Loop(1, 3)
    elif Q < Mc:
        return MAlphaS2Loop(Mc, 4) * MAlphaS2Loop(Mb, 5) * AlphaSNf3Loop(Q, 3)
    elif Q < Mb:
        return MAlphaS2Loop(Mb, 5) * AlphaSNf3Loop(Q, 4)
    elif Q < Mt:
        return AlphaSNf3Loop(Q, 5)
    else: # Q >= Mt
        return (1 / MAlphaS2Loop(Mt, 6)) * AlphaSNf3Loop(Q, 6)

def AlphaS2Loop(Q):
    if Q < 1:
        return AlphaSNf2Loop(1, 3)
    elif Q < Mc:
        return AlphaSNf2Loop(Q, 3)
    elif Q < Mb:
        return AlphaSNf2Loop(Q, 4)
    elif Q < Mt:
        return AlphaSNf2Loop(Q, 5)
    else: # Q >= Mt
        return AlphaSNf2Loop(Q, 6)

def AlphaS1Loop(Q):
    if Q < 1:
        return AlphaSNf1Loop(1, 3)
    elif Q < Mc:
        return AlphaSNf1Loop(Q, 3)
    elif Q < Mb:
        return AlphaSNf1Loop(Q, 4)
    elif Q < Mt:
        return AlphaSNf1Loop(Q, 5)
    else: # Q >= Mt
        return AlphaSNf1Loop(Q, 6)


### Tests:

In [7]:
print(
    AlphaS(2),'\n', #In mathematica, the result is : 0.293951
    AlphaS3Loop(2), '\n', #In mathematica, the result is : 0.30283
    AlphaS2Loop(2), '\n', #In mathematica, the result is : 0.2923
    AlphaS1Loop(2)) #In mathematica, the result is : 0.394643

0.33509972365551155 
 0.3028107299035 
 0.29230005539156734 
 0.3946429024640088


In [8]:
MZ = 91.1876
AlphaS(MZ) #In mathematica, the result is : 0.118288
#In the literrature, I've found AlphaS(Mz) = 0.1181(11). It thus overlap the 2 results

0.11869454756931111